In [4]:
import cv2
import os

In [5]:
def extract_frames(video_path, output_folder, save_name = None, frames_per_sec=None, start_time_sec=None, end_time_sec=None):
    """
    Extracts frames from a video at a specified rate and within a time interval.

    Args:
        video_path (str): The path to the video file.
        output_folder (str): The path to the folder where frames will be saved.
        frames_per_sec (int, optional): The number of frames to extract per second.
                                        If None, extracts all frames. Defaults to None.
        start_time_sec (int, optional): The start time in seconds. Defaults to None.
        end_time_sec (int, optional): The end time in seconds. Defaults to None.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file at {video_path}")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    # print(f"Original FPS:{original_fps}")
    
    frame_skip_interval = 1
    if frames_per_sec and frames_per_sec > 0:
        frame_skip_interval = round(original_fps / frames_per_sec)

    frame_count = 0
    saved_frame_count = 0
    
    while True:
        success, frame = cap.read()
        if not success:
            break

        current_msec = cap.get(cv2.CAP_PROP_POS_MSEC)
        current_sec = current_msec / 1000.0

        if start_time_sec and current_sec < start_time_sec:
            frame_count += 1
            continue
        if end_time_sec and current_sec > end_time_sec:
            break

        if frame_count % frame_skip_interval == 0:
            frame_filename = f"frame_{saved_frame_count:04d}.jpg"
            if save_name is not None and isinstance(save_name, str):
                frame_filename = save_name + frame_filename
            save_path = os.path.join(output_folder, frame_filename)
            cv2.imwrite(save_path, frame)
            saved_frame_count += 1

        frame_count += 1

    cap.release()
    print(f"Successfully extracted {saved_frame_count} frames to {output_folder}")

In [6]:
VIDEO_PATH = '../cat_action_videos/'
video_list = os.listdir(VIDEO_PATH)
print(video_list)

['.DS_Store', 'downloaded_videos.json', 'testing_raw', 'scratching_raw', 'playing', 'scratching', 'testing', 'frames.json', 'processed_video_names.json']


Process VideoName

In [5]:
ACTION_NAME = 'playing'
import json

# Find existing videos to continue indexing
organized_folder = os.path.join(VIDEO_PATH, ACTION_NAME)
os.makedirs(organized_folder, exist_ok=True)

# Get highest existing index
existing_videos = [f for f in os.listdir(organized_folder) if f.endswith('.mp4')]
if existing_videos:
    existing_indices = []
    for video in existing_videos:
        if video.startswith(ACTION_NAME + '_'):
            try:
                idx = int(video.split('_')[1].split('.')[0])
                existing_indices.append(idx)
            except:
                pass
    index = max(existing_indices) + 1 if existing_indices else 0
else:
    index = 0

old_base_path = organized_folder + '_raw'
processed_count = 0

# Track original video names to prevent duplicate processing
processed_names_file = os.path.join(VIDEO_PATH, 'processed_video_names.json')
if os.path.exists(processed_names_file):
    with open(processed_names_file, 'r') as f:
        processed_names = json.load(f)
else:
    processed_names = []

for video in os.listdir(old_base_path):
    if not video.endswith('.mp4'): 
        continue
    
    if video in processed_names:
        print(f'Skipping {video} - already processed')
        continue
    
    video_old_path = os.path.join(old_base_path, video)
    video_new_path = os.path.join(organized_folder, f'{ACTION_NAME}_{index}.mp4')
    
    if not os.path.exists(video_new_path):
        os.rename(video_old_path, video_new_path)
        print(f'Renamed: {video} -> {ACTION_NAME}_{index}.mp4')
        processed_names.append(video)
        index += 1
        processed_count += 1

with open(processed_names_file, 'w') as f:
    json.dump(processed_names, f, indent=2)

print(f'Video renaming complete. Processed {processed_count} new videos. Next index: {index}')

Renamed: Ballpool Obstacle Course Challenge [tzcamFb21Ro].mp4 -> playing_0.mp4
Renamed: Funny Cat Pulls Off Sneak Attack! [EHUqAyYlhxY].mp4 -> playing_1.mp4
Renamed: 🐱 Funny cat videos ｜ cute cats ｜ Try not to laugh ｜ Cat videos Compilation #shorts  🐈 [1UgJI6O8T2U].mp4 -> playing_2.mp4
Renamed: Silliest CATS on Earth 😂   Funny Cats Videos 2023 [RFMuw3xpmFE].mp4 -> playing_3.mp4
Renamed: Cat Barsik Beads 📿 Domino Reverse Video #cat #catvideos #asatisfyingvideofamily #funny [NSWihO8N8W8].mp4 -> playing_4.mp4
Renamed: Cats Videos For Cats To Watch With Sound ➙ EPIC 3 HOURS! ＊ Cats Playing ＊ Entertainment For Cats [blsWaepdH80].mp4 -> playing_5.mp4
Renamed: 20 Minutes of Adorable Kittens 😍 ｜ BEST Compilation [y0sF5xhGreA].mp4 -> playing_6.mp4
Renamed: A Cat With Infinite Rizz ｜ Funny Cat Compilation - FailArmy [DHfRfU3XUEo].mp4 -> playing_7.mp4
Renamed: Cats Videos For Cats To Watch ➙ EPIC 8 HOURS With Sound! ＊ Cats Playing ＊ Entertainment For Cats [pP-pPlS7D7k].mp4 -> playing_8.mp4
Rename

Parse Json format config file for extracting frames in simple loop

In [9]:
import json
# !pwd
CONFIG_PATH = '../cat_action_videos/frames.json'
with open(CONFIG_PATH, 'r') as f:
    frame_json = json.load(f)
print((frame_json['playing']['0']))

[8, 90, 96]


Test frames_extraction

In [10]:
# test_video_path = os.path.join(video_path, ACTION_NAME, video_list[3])
# extract_frames(video_path=test_video_path, output_folder='../cat_video_frames/scratching/video3', save_name=None,
#                frames_per_sec=8, start_time_sec=3 , end_time_sec=16)

Frame Extractions simple approach

In [11]:
# Always find all the videos and work on frames extraction, not time-consuming
# 需要提前删除所有clip文件夹

# ACTION_LIST = list(frame_json.keys())
video_clip_index = 0
processed_clips = 0

for action in frame_json.keys():
    for id in frame_json[action].keys():
        video_path = os.path.join(VIDEO_PATH, action, f'{action}_{id}.mp4')
        if not os.path.exists(video_path):
            print(f"[!] Video not found at {video_path}")
            continue
            
        info = frame_json[action][id]
        if (len(info) - 1) % 2 != 0: 
            print(f"[!] Skip wrong format annotation: action {action}, video id {id}")
            continue
            
        frame_per_second = info[0]
        clip_pairs = int((len(info) - 1) / 2)
        
        for i in range(clip_pairs):
            clip_output_dir = f'../cat_video_frames/{action}/clip_{video_clip_index}'
            
            # Skip if clip already exists and has frames
            if os.path.exists(clip_output_dir) and len(os.listdir(clip_output_dir)) > 0:
                print(f"Clip {video_clip_index} already exists, skipping...")
                video_clip_index += 1
                continue
                
            start_time = info[i*2 + 1]
            end_time = info[i*2 + 2]
            
            extract_frames(video_path, clip_output_dir, 
                         frames_per_sec=frame_per_second, 
                         start_time_sec=start_time, end_time_sec=end_time)
            
            video_clip_index += 1
            processed_clips += 1

print(f"Frame extraction complete. Processed {processed_clips} new clips. Next clip index: {video_clip_index}")

Clip 0 already exists, skipping...
Successfully extracted 0 frames to ../cat_video_frames/scratching/clip_1
Clip 2 already exists, skipping...
Clip 3 already exists, skipping...
Clip 4 already exists, skipping...
Clip 5 already exists, skipping...
Clip 6 already exists, skipping...
Clip 7 already exists, skipping...
Clip 8 already exists, skipping...
Clip 9 already exists, skipping...
Clip 10 already exists, skipping...
Clip 11 already exists, skipping...
Clip 12 already exists, skipping...
Successfully extracted 0 frames to ../cat_video_frames/scratching/clip_13
Clip 14 already exists, skipping...
Clip 15 already exists, skipping...
Clip 16 already exists, skipping...
Successfully extracted 0 frames to ../cat_video_frames/scratching/clip_17
Successfully extracted 0 frames to ../cat_video_frames/scratching/clip_18
Successfully extracted 0 frames to ../cat_video_frames/scratching/clip_19
Clip 20 already exists, skipping...
Successfully extracted 0 frames to ../cat_video_frames/scratchin